In [1]:
import torch
import torchvision
import pretrained
from actuallysparse import converter

In [2]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset_train = torchvision.datasets.CIFAR10(".data", download=True, transform=transform)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=16)
dataset_test = torchvision.datasets.CIFAR10(".data", download=True, train=False, transform=transform)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=16)
training_device = "cpu"

Files already downloaded and verified
Files already downloaded and verified


In [3]:
very_pruned = torch.load(".weights/full/very_pruned").to(training_device).eval()

In [4]:
import copy
with torch.no_grad():
    very_pruned_coo = copy.deepcopy(very_pruned)

    very_pruned_coo.classifier[0] =  converter.convert(very_pruned_coo.classifier[0], "coo")
    very_pruned_coo.classifier[3] =  converter.convert(very_pruned_coo.classifier[3], "coo")
    # very_pruned_coo.classifier[6] =  converter.convert(very_pruned_coo.classifier[6], "coo")

In [5]:
for inputs, labels in dataloader_test:
    break

In [6]:
test_input =  inputs[1:3, :]

In [7]:
very_pruned(test_input)

tensor([[0.3092, 0.5177, 0.1881, 0.1132, 1.0875, 0.2068, 0.2365, 0.0506, 1.9462,
         0.9583],
        [1.1969, 0.4866, 0.3137, 0.1361, 1.0605, 0.1905, 0.2581, 0.0570, 1.7939,
         0.8596]], grad_fn=<AddmmBackward0>)

In [8]:
very_pruned_coo(test_input)


tensor([[0.3092, 0.5177, 0.1881, 0.1132, 1.0875, 0.2068, 0.2365, 0.0506, 1.9462,
         0.9583],
        [1.1969, 0.4866, 0.3137, 0.1361, 1.0605, 0.1905, 0.2581, 0.0570, 1.7939,
         0.8596]], grad_fn=<AddmmBackward0>)

In [9]:
test_feature = very_pruned.features(test_input).view(very_pruned.features(test_input).size(0), -1)

In [28]:
with torch.no_grad():
    x = test_feature
    x = very_pruned.classifier[0](x)
    x = very_pruned.classifier[1](x)
    x = very_pruned.classifier[2](x)
    x = very_pruned.classifier[3](x)

x

tensor([[0.0564, 0.0000, 0.0690,  ..., 0.0000, 0.0295, 0.0000],
        [0.2134, 0.0000, 0.0660,  ..., 0.0000, 0.0250, 0.0000]])

In [29]:
very_pruned_coo.classifier[3].weight.data = very_pruned.classifier[3].weight.to_sparse_coo()

In [30]:
with torch.no_grad():
    x = test_feature
    x = very_pruned_coo.classifier[0](x)
    x = very_pruned_coo.classifier[1](x)
    x = very_pruned_coo.classifier[2](x)
    x = very_pruned_coo.classifier[3](x)

x

tensor([[0.0168, 0.0000, 0.1172,  ..., 0.0802, 0.0184, 0.0000],
        [0.0280, 0.0000, 0.1167,  ..., 0.1072, 0.0247, 0.0000]])

In [31]:
very_pruned_coo.classifier[0]

SparseLayer(in_features=512, out_features=4096, bias=True, csr_mode=False, k=0.05)

In [32]:
very_pruned.classifier[3]

Linear(in_features=4096, out_features=4096, bias=True)

In [40]:
very_pruned.classifier[3].forward(torch.ones(1, 4096))

tensor([[0.2948, 0.0000, 0.0310,  ..., 0.0000, 0.1325, 0.0000]],
       grad_fn=<AddmmBackward0>)

In [44]:
converter.convert(very_pruned.classifier[3], "coo").forward(torch.ones(1, 4096))

tensor([[0.1524, 0.0000, 0.3816,  ..., 0.4300, 0.2164, 0.0000]],
       grad_fn=<AddBackward0>)